In [ ]:
import os 
from glob import glob
import subprocess

In [28]:
def removefiles(files):
    for fi in files:
        os.remove(fi)

In [72]:
fpath = "/media/ljp238/12TBWolf/ARCHIEVE/FB_CHM/LMK/*.tif"
files = glob(fpath); print(len(files))
#removefiles(files)
m1files = [i for i in files if i.endswith('masked.tif')];print(len(m1files))
m2files = [i for i in files if 'masked.tif' not in i];print(len(m1files))

84
42
42


In [ ]:


def filelist2txt(filelist, txt):
    """Writes a list of file paths to a text file."""
    with open(txt, 'w') as f:
        f.writelines(f"{fi}\n" for fi in filelist)

def buildvrt(txt, vrt=None, epsg='4749'):
    """Builds a VRT file using GDAL from a text file containing file paths."""
    if vrt is None:
        vrt = txt.replace('.txt', '.vrt')

    if epsg is None:
        cmd = ['gdalbuildvrt', '-input_file_list', txt, vrt]
    else:
        cmd = ['gdalbuildvrt', '-a_srs', f"EPSG:{epsg}", '-input_file_list', txt, vrt]

    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: gdalbuildvrt failed with error code {e.returncode}")

def filelist2vrt(filelist, txt):
    """Creates a VRT file from a list of file paths."""
    filelist2txt(filelist, txt)
    buildvrt(txt)


In [61]:
VRT_dpath = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT"
# 4979 4326

In [62]:
FB_CHM_masked_txt = os.path.join(VRT_dpath, 'FB_CHM_masked.txt')
FB_CHM_txt = os.path.join(VRT_dpath, 'FB_CHM.txt')

filelist2vrt(m1files, FB_CHM_masked_txt)
filelist2vrt(m2files, FB_CHM_txt)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [69]:
from osgeo import gdal

def resample_raster(input_raster, output_raster, algo ='nearest', resolution=1/3600, epsg=4979):
    """
    Resamples a raster to a specified resolution and reprojects it to a given EPSG.

    Parameters:
    - input_raster (str): Path to the input raster file.
    - output_raster (str): Path to save the output resampled raster.
    - resolution (float): Output resolution in degrees (default: 1/3600).
    - epsg (int): EPSG code for the target projection (default: 4947).

    Returns:
    - None
    """
    output_raster = output_raster.replace('.tif', f'{algo}.tif')
    if os.path.isfile(output_raster):
        return None

    options = gdal.WarpOptions(
        format='GTiff',  # Output format (GeoTIFF)
        xRes=resolution,  # X resolution
        yRes=resolution,  # Y resolution
        dstSRS=f"EPSG:{epsg}",  # Target coordinate system
        resampleAlg=algo,# Resampling method (change to 'cubic', 'nearest' if needed) 'bilinear
    )
    
    gdal.Warp(output_raster, input_raster, options=options)


In [64]:
m1files = [i for i in files if i.endswith('masked.tif')];print(len(m1files))
m2files = [i for i in files if 'masked.tif' not in i];print(len(m1files))

42
42


In [65]:
dout = "/media/ljp238/12TBWolf/ARCHIEVE/FB_CHM/RESAMPLE/Ymasked/"

In [ ]:
for i, fi in enumerate(m1files):
    if i > 0: break
    fo = os.path.join(dout, os.path.basename(fi))
    for algo in ['nearest','bilinear']:
        resample_raster(fi, fo,algo =algo)
    